In [1]:
#  word level CTC  tensorflow version 1.3

# this module requires the following :

#  a dataset csv file ( have a look at dataset csv generator)
#  a dictionary.txt file ( have a look at dictionary folder)



# to do :   
#   , get rid of fixed batch size , instead use variable batch size  


# perform following tests :

# 1. variable batch sizes test
# 2. GPU test





# please change the following according to your system
# hyperparameters in this notebook

# change paths in this notebook to that corresponding to your system
# please note number of audio examples in dataset must be even 

# peace.




In [2]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator
import time


/home/saurabh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate, nfilt=55,numcep=15)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)


In [5]:
targets={}
# Constants
#SPACE_TOKEN = '<space>'
SPACE_INDEX = 0
#FIRST_INDEX = ord('a') - 96  # 0 is reserved to space


In [6]:
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [7]:
inputs.shape

(299, 15)

In [8]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
    
       
       

       return parsed_line[0] , parsed_line[1]

In [9]:
# we also need a fixed vocabulary 
import re

word_dictionary = {}

with open("words/hardik_words.txt") as file:
    for i , line in enumerate(file):
        
        line = line.replace("\n", "")
        word_dictionary[line] = i



In [10]:
print(len(word_dictionary))

1303


In [11]:
# this function is required for CTC Loss
# for it's input , first convert transcrition / ground truth to number representation 

def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

In [12]:

def sentence_to_char_array(sentence):
    # convert sentence to lowercase and remove numbers
    sentence=sentence.lower()
    # remove anything that's not a-z 
    sentence=re.sub('[^a-z ]+', '', sentence)
    
    targets = np.hstack([SPACE_TOKEN if x == '' else list(x) for x in sentence])

    # Transform char into index
    targets = np.asarray([SPACE_INDEX if x == ' ' else ord(x) - 96
                      for x in targets])


    # Creating sparse representation to feed the placeholder
    #train_targets = sparse_tuple_from([targets])

    
    
    return targets




In [13]:
sentence_to_char_array('this is a test ')

array([20,  8,  9, 19,  0,  9, 19,  0,  1,  0, 20,  5, 19, 20,  0])

In [14]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]

In [15]:
def _read_py_function(audio, label):
    audio = audio_to_mfcc(audio)
    
    if audio.shape[0] < timesteps:
        original_length=audio.shape[0]
       # print(original_length)
    
    elif audio.shape[0] >= timesteps:
        original_length=timesteps
        
    #audio=normalized(pad(audio) , axis=1 )
    audio=pad(audio)
   
    return audio ,label, original_length

In [16]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_orange/tf_orange/hardik.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype, tf.int64])))




dataset=dataset.batch(4)


Instructions for updating:
Use `tf.data.TextLineDataset`.


In [17]:

iterator = dataset.make_initializable_iterator()

item = iterator.get_next()

In [18]:
# Training Parameters
learning_rate = 0.01
training_steps = 100
#batch_size = 2
display_step = 200
num_features = 15
beta=0.01
logs_path = '/home/saurabh/Documents/tf_orange/tf_orange/hardik/logs'
model_path = '/home/saurabh/Documents/tf_orange/tf_orange/models/hardik/lite/model.ckpt'

# Network Parameters

timesteps = 300 # timesteps
num_hidden = 128 # hidden layer num of features

# Accounting the 0th indice +  space + blank symbol  = 27 characters
num_classes = ord('z') - ord('a') + 1 + 1 + 1




In [19]:
#prediction = tf.nn.softmax(logits)
inputs = tf.placeholder(tf.float32, [None, None, num_features])
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])
batch_size = 4

#batch_size = tf.placeholder ( tf.int32 )


In [20]:

# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([  2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]), name='bias')
}


In [21]:
'''
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([  num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]), name='bias')
}
'''

"\n# Define weights\nweights = {\n    'out': tf.Variable(tf.random_normal([  num_hidden, num_classes]))\n}\nbiases = {\n    'out': tf.Variable(tf.random_normal([num_classes]), name='bias')\n}\n"

In [22]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    outputs=tf.reshape(outputs, [-1,2*num_hidden])
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    res = tf.reshape(res, [batch_size,timesteps,num_classes])
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [23]:
'''def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
   # x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    

    
    # Get lstm cell output
   # outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell , x,
    #dtype=tf.float32)
    
    outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,
                                   inputs=x,
                                   sequence_length=seq_len,
                                   dtype=tf.float32)
    
    
    #convert output shape (timesteps * batch * hidden units ) to (batch*timesteps* hidden units)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    outputs=tf.reshape(outputs, [-1,num_hidden])
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    res = tf.reshape(res, [batch_size,timesteps,num_classes])
    
    # new code 
    # state is of shape ( batch size ,  hidden_units)
    # need to adapt weights accordingly 
    
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out']) '''

"def RNN(x, weights, biases):\n\n    # Prepare data shape to match `rnn` function requirements\n    # Current data input shape: (batch_size, timesteps, n_input)\n    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)\n\n    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)\n   # x = tf.unstack(x, timesteps, 1)\n\n    # Define a lstm cell with tensorflow\n    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)\n    \n    \n\n    \n    # Get lstm cell output\n   # outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell , x,\n    #dtype=tf.float32)\n    \n    outputs, state = tf.nn.dynamic_rnn(cell=lstm_cell,\n                                   inputs=x,\n                                   sequence_length=seq_len,\n                                   dtype=tf.float32)\n    \n    \n    #convert output shape (timesteps * batch * hidden units ) to (batch*timesteps* hidden units)\n    outputs=tf.transpose( outputs , [1, 0, 2])\n    \n   

In [24]:




logits = RNN(inputs, weights, biases)

loss =  tf.nn.ctc_loss ( targets, logits , seq_len , time_major = False)
cost = tf.reduce_mean(loss)

tv = tf.trainable_variables()
variables_names = [v.name for v in tf.trainable_variables()]

reg = tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv if 'bias' not in v.name])

total_loss = tf.reduce_mean(cost + reg * beta)

optimizer = tf.train.MomentumOptimizer(learning_rate,
                                           0.9).minimize(total_loss)

gradient_optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)

adam_optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)


# Option 2: tf.contrib.ctc.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
decoder_input = tf.transpose(logits, [1, 0, 2])

decoded, log_prob = tf.nn.ctc_greedy_decoder(decoder_input, seq_len)

    

In [25]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [26]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [27]:
init = tf.global_variables_initializer()

In [28]:
# need this for decoding word give its index key value 

def keys_of_value(dct, value):
    for k in dct:
        if isinstance(dct[k], list):
            if value in dct[k]:
                return k
        else:
            if value == dct[k]:
                return k

In [29]:
# testing above function 

print(keys_of_value(word_dictionary, 2419))

None


In [ ]:
# retrain from a checkpoint
# if training first time, ignore this block

with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)
        
    # Restore model weights from previously saved model
    load_path = saver.restore(sess, model_path +  '/2' )
    print("Model restored from file: %s" % load_path)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_steps):
     #   avg_cost = 0.
           # Save model weights to disk
        save_path = saver.save(sess, model_path +  '/2')
        print("Model saved in file: %s" % save_path) 
        
        
        sess.run(iterator.initializer)
        train_cost=0
        
        while True:
            try:
                Next_element=sess.run(item)
             
            
                 
                print(Next_element[1])   
                
                batch_list = []
                
 
                for index,j in enumerate(Next_element[1]):
                    batch_list.insert(index, )
                    
                
                batch_targets = sparse_tuple_from(batch_list)
                
                print(Next_element[2])
                
                feed = {inputs: Next_element[0],
                        targets: batch_targets,
                        seq_len: Next_element[2],
                         }
	    

                batch_cost, _ , summary = sess.run([cost, adam_optimizer , merged_summary_op ], feed)
               # print(batch_cost)
                
                train_cost += batch_cost*batch_size
                
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch )
                
                 # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
                for i, seq in enumerate(dense_decoded):

                    seq = [s for s in seq if s != -1]
       
                    #str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in seq ])
                    #print(str_decoded)
            
              
            except tf.errors.OutOfRangeError:
              print(train_cost)
              
              
              break
                
        # Save model weights to disk at the end of all epochs, if we remove this last epoch doesnt gets saved
    save_path = saver.save(sess, model_path +  '/2')
    print("Model saved in file: %s" % save_path)        
    
    

In [30]:
# Start training
with tf.Session() as sess:
    sess.run(init)
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_steps):
     #   avg_cost = 0.
           # Save model weights to disk
        save_path = saver.save(sess, model_path)
        print("Model saved in file: %s" % save_path) 
        
        
        sess.run(iterator.initializer)
        train_cost=0
        
        while True:
            try:
                Next_element=sess.run(item)
             
            
                 
                print(Next_element[1])   
                
                #batch_list=np.zeros(2)
                
                batch_list = []
                
 
                for index,j in enumerate(Next_element[1]):
                    batch_list.insert(index, sentence_to_char_array(j.decode("utf-8")))
                    
                
                batch_targets = sparse_tuple_from(batch_list)
                
                #batch_list =  sentence_to_char_array(Next_element[1][0].decode("utf-8"))
                    
                
                #batch_targets = sparse_tuple_from(batch_list)
                print(batch_list)
                
                print(Next_element[2])
                
                feed = {inputs: Next_element[0],
                        targets: batch_targets,
                        seq_len: Next_element[2],
                         }
	    

                batch_cost, _ , summary = sess.run([cost, optimizer , merged_summary_op ], feed)
               # print(batch_cost)
                
                train_cost += batch_cost*batch_size
                
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch )
                
                 # Decoding
                d = sess.run(decoded[0], feed_dict=feed)
   
                dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
                for i, seq in enumerate(dense_decoded):

                    seq = [s for s in seq if s != -1]
                
                    #print(seq)
       
                    str_decoded = ''.join([ chr(x+96)  for x in seq ])
                    print('decoded :' + str_decoded)
            
              
            except tf.errors.OutOfRangeError:
              print(train_cost)
              
              
              break
                
        # Save model weights to disk at the end of all epochs, if we remove this last epoch doesnt gets saved
    save_path = saver.save(sess, model_path)
    print("Model saved in file: %s" % save_path)         

               
        
            
                
                
             
                
                
            

Model saved in file: /home/saurabh/Documents/tf_orange/tf_orange/models/hardik/lite/model.ckpt
[b'Flexible battery inspired by human spine'
 b'Amateur radio astronomer discovers long-lost satellite'
 b'community members visit India' b'Moderate Blood Sugar Levels']
[array([ 6, 12,  5, 24,  9,  2, 12,  5,  0,  2,  1, 20, 20,  5, 18, 25,  0,
        9, 14, 19, 16,  9, 18,  5,  4,  0,  2, 25,  0,  8, 21, 13,  1, 14,
        0, 19, 16,  9, 14,  5]), array([ 1, 13,  1, 20,  5, 21, 18,  0, 18,  1,  4,  9, 15,  0,  1, 19, 20,
       18, 15, 14, 15, 13,  5, 18,  0,  4,  9, 19,  3, 15, 22,  5, 18, 19,
        0, 12, 15, 14,  7, 12, 15, 19, 20,  0, 19,  1, 20,  5, 12, 12,  9,
       20,  5]), array([ 3, 15, 13, 13, 21, 14,  9, 20, 25,  0, 13,  5, 13,  2,  5, 18, 19,
        0, 22,  9, 19,  9, 20,  0,  9, 14,  4,  9,  1]), array([13, 15,  4,  5, 18,  1, 20,  5,  0,  2, 12, 15, 15,  4,  0, 19, 21,
        7,  1, 18,  0, 12,  5, 22,  5, 12, 19])]
[300 300 300 294]
decoded :raoano`o`o`o`n`n`n`o`o`oeo

decoded :`at`at`a`h`adta
decoded :hqt``v`hvkvs
decoded :d`t```vqs`t
decoded :tp`qtf`t`t`d``m`
[b'no winners when the die' b"it's the final dance" b"So don't give up"
 b"just young lovers' romance"]
[array([14, 15,  0, 23,  9, 14, 14,  5, 18, 19,  0, 23,  8,  5, 14,  0, 20,
        8,  5,  0,  4,  9,  5]), array([ 9, 20, 19,  0, 20,  8,  5,  0,  6,  9, 14,  1, 12,  0,  4,  1, 14,
        3,  5]), array([19, 15,  0,  4, 15, 14, 20,  0,  7,  9, 22,  5,  0, 21, 16]), array([10, 21, 19, 20,  0, 25, 15, 21, 14,  7,  0, 12, 15, 22,  5, 18, 19,
        0, 18, 15, 13,  1, 14,  3,  5])]
[247 213 182 300]
decoded :tst``l`t`a`f`ntafnstf
decoded :sqltmr``s`b```s`lqt`t`tmu
decoded :dlds```t`t`u``r`s`s`tm`tds
decoded :`sts`a```
[b'Keep you to Winter' b'Alarming increase in tobacco related cancer'
 b"E-cigarettes 'should be on prescription'" b'Disturbing trend']
[array([11,  5,  5, 16,  0, 25, 15, 21,  0, 20, 15,  0, 23,  9, 14, 20,  5,
       18]), array([ 1, 12,  1, 18, 13,  9, 14,  7,  0,  9, 14,  

decoded :tdkkboomogvgvnntnttottvngtvmm
decoded :epneeteettt
decoded :ogjtnnnglntntnnntknk
decoded :pmggtggnathoee
[b'Chinese shipping firm MD killed'
 b'Amateur radio astronomer discovers long-lost satellite'
 b'Indian-Americans hold rally outside White House'
 b'Chinese Incursions Into India Rose']
[array([ 3,  8,  9, 14,  5, 19,  5,  0, 19,  8,  9, 16, 16,  9, 14,  7,  0,
        6,  9, 18, 13,  0, 13,  4,  0, 11,  9, 12, 12,  5,  4]), array([ 1, 13,  1, 20,  5, 21, 18,  0, 18,  1,  4,  9, 15,  0,  1, 19, 20,
       18, 15, 14, 15, 13,  5, 18,  0,  4,  9, 19,  3, 15, 22,  5, 18, 19,
        0, 12, 15, 14,  7, 12, 15, 19, 20,  0, 19,  1, 20,  5, 12, 12,  9,
       20,  5]), array([ 9, 14,  4,  9,  1, 14,  1, 13,  5, 18,  9,  3,  1, 14, 19,  0,  8,
       15, 12,  4,  0, 18,  1, 12, 12, 25,  0, 15, 21, 20, 19,  9,  4,  5,
        0, 23,  8,  9, 20,  5,  0,  8, 15, 21, 19,  5]), array([ 3,  8,  9, 14,  5, 19,  5,  0,  9, 14,  3, 21, 18, 19,  9, 15, 14,
       19,  0,  9, 14, 20, 15,  0,

decoded :fpo`ooaoyyayayyac`
decoded :eleler```e`ic
decoded :eslrcwle`oa`
decoded :rorllemoos``heeeeeo````
[b'gears up for India expansion' b'Mercedes Benz C Class Revealed'
 b'Samsung Galaxy S9 to Feature 3D Emoji'
 b"Indian scientists develop world's thinnest material with novel technique"]
[array([ 7,  5,  1, 18, 19,  0, 21, 16,  0,  6, 15, 18,  0,  9, 14,  4,  9,
        1,  0,  5, 24, 16,  1, 14, 19,  9, 15, 14]), array([13,  5, 18,  3,  5,  4,  5, 19,  0,  2,  5, 14, 26,  0,  3,  0,  3,
       12,  1, 19, 19,  0, 18,  5, 22,  5,  1, 12,  5,  4]), array([19,  1, 13, 19, 21, 14,  7,  0,  7,  1, 12,  1, 24, 25,  0, 19,  0,
       20, 15,  0,  6,  5,  1, 20, 21, 18,  5,  0,  4,  0,  5, 13, 15, 10,
        9]), array([ 9, 14,  4,  9,  1, 14,  0, 19,  3,  9,  5, 14, 20,  9, 19, 20, 19,
        0,  4,  5, 22,  5, 12, 15, 16,  0, 23, 15, 18, 12,  4, 19,  0, 20,
        8,  9, 14, 14,  5, 19, 20,  0, 13,  1, 20,  5, 18,  9,  1, 12,  0,
       23,  9, 20,  8,  0, 14, 15, 22,  5, 12,  0, 20,

decoded :agaoensfn`n``eeenofopusudsf
decoded :ononsmomomucereiprppascsicu
decoded :ananas`sesooooindirncnmmufiif`s`bsdolnniue
decoded :snngsscs`snssdvsdy`lsasbeuvasaea``
[b'Meditation has limited role in making you a better person'
 b'Domestic Abuse and the White House' b'body found in bed box'
 b'warns youth from celebrating']
[array([13,  5,  4,  9, 20,  1, 20,  9, 15, 14,  0,  8,  1, 19,  0, 12,  9,
       13,  9, 20,  5,  4,  0, 18, 15, 12,  5,  0,  9, 14,  0, 13,  1, 11,
        9, 14,  7,  0, 25, 15, 21,  0,  1,  0,  2,  5, 20, 20,  5, 18,  0,
       16,  5, 18, 19, 15, 14]), array([ 4, 15, 13,  5, 19, 20,  9,  3,  0,  1,  2, 21, 19,  5,  0,  1, 14,
        4,  0, 20,  8,  5,  0, 23,  8,  9, 20,  5,  0,  8, 15, 21, 19,  5]), array([ 2, 15,  4, 25,  0,  6, 15, 21, 14,  4,  0,  9, 14,  0,  2,  5,  4,
        0,  2, 15, 24]), array([23,  1, 18, 14, 19,  0, 25, 15, 21, 20,  8,  0,  6, 18, 15, 13,  0,
        3,  5, 12,  5,  2, 18,  1, 20,  9, 14,  7])]
[300 263 263 300]
decoded :inbi

...

decoded :ko`mmhnunu`kolomo`os`non`nimmnmmmno`noion``mmaimosih`sn`inh
decoded :minmnmim``````t```n
decoded :ohmnmninininmpm`n`in`rwm
decoded :osfbmitnaitwiiisi`oninngtniwioo
[b"things I don't ask" b'I know what it takes' b'to fool this town'
 b'the sun goes down']
[array([20,  8,  9, 14,  7, 19,  0,  9,  0,  4, 15, 14, 20,  0,  1, 19, 11]), array([ 9,  0, 11, 14, 15, 23,  0, 23,  8,  1, 20,  0,  9, 20,  0, 20,  1,
       11,  5, 19]), array([20, 15,  0,  6, 15, 15, 12,  0, 20,  8,  9, 19,  0, 20, 15, 23, 14]), array([20,  8,  5,  0, 19, 21, 14,  0,  7, 15,  5, 19,  0,  4, 15, 23, 14])]
[231 213 213 198]
decoded :`oiobgwhpmghnnbnn`nitsin`n`n`nnooooit
decoded :oi```ftfawo``oy
decoded :oi`im`ioot`i```onk`oikiniitnin`
decoded :niimia`i`mnnnioiilhignofnwwnnhigs
[b'all through the night time' b'what you wanna hear'
 b'Leave my sunglasses on' b'put my armor on']
[array([ 1, 12, 12,  0, 20,  8, 18, 15, 21,  7,  8,  0, 20,  8,  5,  0, 14,
        9,  7,  8, 20,  0, 20,  9, 13,  5]), array([23,  

decoded :daadyy
decoded :huiaanomadmamna`aa`namn
decoded :fwt``yt`odbo
decoded :aohn`n``iatat``t
[b"China's success in cloning monkeys was a victory for hard work"
 b'But Im too afraid now'
 b'Cancer Treatment Could Increase Heart Disease Risk'
 b'ill kids loved puts life into perspective']
[array([ 3,  8,  9, 14,  1, 19,  0, 19, 21,  3,  3,  5, 19, 19,  0,  9, 14,
        0,  3, 12, 15, 14,  9, 14,  7,  0, 13, 15, 14, 11,  5, 25, 19,  0,
       23,  1, 19,  0,  1,  0, 22,  9,  3, 20, 15, 18, 25,  0,  6, 15, 18,
        0,  8,  1, 18,  4,  0, 23, 15, 18, 11]), array([ 2, 21, 20,  0,  9, 13,  0, 20, 15, 15,  0,  1,  6, 18,  1,  9,  4,
        0, 14, 15, 23]), array([ 3,  1, 14,  3,  5, 18,  0, 20, 18,  5,  1, 20, 13,  5, 14, 20,  0,
        3, 15, 21, 12,  4,  0,  9, 14,  3, 18,  5,  1, 19,  5,  0,  8,  5,
        1, 18, 20,  0,  4,  9, 19,  5,  1, 19,  5,  0, 18,  9, 19, 11]), array([ 9, 12, 12,  0, 11,  9,  4, 19,  0, 12, 15, 22,  5,  4,  0, 16, 21,
       20, 19,  0, 12,  9,  6,  5, 

decoded :afeon`f`npkp`fr`e`nrinrmni`tdntdannddnaaca
decoded :lltek```s`n`knwnpkrarnr`o`n`uutvlke
decoded :natlupkp`ngptcteteteti`pp`v`tace`ke`eebonpr`ra``nknbdabd
decoded :ceaka
[b'Scariest Flight Of My Life' b'bet a million on you'
 b'no way around this' b"only way now it's through"]
[array([19,  3,  1, 18,  9,  5, 19, 20,  0,  6, 12,  9,  7,  8, 20,  0, 15,
        6,  0, 13, 25,  0, 12,  9,  6,  5]), array([ 2,  5, 20,  0,  1,  0, 13,  9, 12, 12,  9, 15, 14,  0, 15, 14,  0,
       25, 15, 21]), array([14, 15,  0, 23,  1, 25,  0,  1, 18, 15, 21, 14,  4,  0, 20,  8,  9,
       19]), array([15, 14, 12, 25,  0, 23,  1, 25,  0, 14, 15, 23,  0,  9, 20, 19,  0,
       20,  8, 18, 15, 21,  7,  8])]
[294 198 198 231]
decoded :ptpt`t`tototot`tnatsv`tlpl`nttlrtf``lt`r``ffl
decoded :tltlt`t`t`t`````fnardm
decoded :itlpaf`o`tfr
decoded :tkfil`t`l`t`nwict`t`r`i`tit`tabrdia
[b'gonna cry myself to sleep' b'kept you to winter'
 b"you won't be needing me" b'not in front of me']
[array([ 7, 15, 14, 14

decoded :ais`soosca``ato`aostststsoa`a`t`t`st`s`
decoded :cpoas```aca`to`as``t
decoded :sasaaot``tr``opoef
decoded :cta`i`a`sap`asesasoas`s`sanco`ecc`c``
[b'diesel at all-time high in Delhi' b'These Two Firms Are In Race'
 b'Niti Aayog health index' b'Trade setup for Tuesday']
[array([ 4,  9,  5, 19,  5, 12,  0,  1, 20,  0,  1, 12, 12, 20,  9, 13,  5,
        0,  8,  9,  7,  8,  0,  9, 14,  0,  4,  5, 12,  8,  9]), array([20,  8,  5, 19,  5,  0, 20, 23, 15,  0,  6,  9, 18, 13, 19,  0,  1,
       18,  5,  0,  9, 14,  0, 18,  1,  3,  5]), array([14,  9, 20,  9,  0,  1,  1, 25, 15,  7,  0,  8,  5,  1, 12, 20,  8,
        0,  9, 14,  4,  5, 24]), array([20, 18,  1,  4,  5,  0, 19,  5, 20, 21, 16,  0,  6, 15, 18,  0, 20,
       21,  5, 19,  4,  1, 25])]
[300 300 300 294]
decoded :an`a`o``t`
decoded :tash`ls`tsn`as`ss`s`s
decoded :aspo`tc`tat`t`taotct``
decoded :csoti`aasss````s`s``sr``t
[b'million stake sale in Tata Technologies'
 b'Bitcoin newbies are getting crushed'
 b'Singtel to invest 

KeyboardInterrupt: 